In [1]:
import numpy as np
from skimage.transform import resize
from keras.applications.xception import Xception, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, RepeatVector, TimeDistributed
import pylidc as pl

Using TensorFlow backend.


In [2]:
feature_extractor = Xception(include_top=False, pooling='max')

In [3]:
num_slices = 64

In [4]:
model = Sequential()
model.add(CuDNNLSTM(400, input_shape=(num_slices, 2048)))
model.add(RepeatVector(num_slices))
model.add(CuDNNLSTM(400, return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
scans = pl.query(pl.Scan)
n = scans.count()

In [6]:
def get_annotated_slices(ann):
    zs = ann.bbox()[-1]
    return zs.start, zs.stop

In [8]:
def get_data(i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.rollaxis(np.array([np.tile(resize(im.pixel_array, (331, 331), mode='constant'),
                                      (3, 1, 1)) for im in images]), 1, 4)
    nodule_zs = set.union(set(), *(set(range(*get_annotated_slices(ann))) for ann in scan.annotations))
    y = np.array([[[i in nodule_zs] for i in range(n)]])
    start = int((n - num_slices) / 2)
    X = X[start:start + num_slices]
    y = y[:, start:start + num_slices]
    return feature_extractor.predict(preprocess_input(X))[None], y

In [9]:
for _ in range(50):
    try:
        i = np.random.randint(n)
        print(i)
        X, y = get_data(i)
        model.fit(X, y, epochs=2)
    except OSError:
        pass

393
Epoch 1/2
1/1 [==============================] - 3s 3s/step - loss: 0.8000 - acc: 0.3438
Epoch 2/2
1/1 [==============================] - 0s 35ms/step - loss: 1.6183 - acc: 0.6562
618
Epoch 1/2
1/1 [==============================] - 3s 3s/step - loss: 0.5217 - acc: 0.8125
Epoch 2/2
1/1 [==============================] - 0s 38ms/step - loss: 0.6000 - acc: 0.8125
252
Epoch 1/2
1/1 [==============================] - 0s 31ms/step - loss: 0.4424 - acc: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 34ms/step - loss: 0.2005 - acc: 1.0000
510
Epoch 1/2
1/1 [==============================] - 0s 31ms/step - loss: 0.2444 - acc: 0.9375
Epoch 2/2
1/1 [==============================] - 0s 41ms/step - loss: 0.2460 - acc: 0.9375
414
Epoch 1/2
1/1 [==============================] - 0s 31ms/step - loss: 0.0331 - acc: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 37ms/step - loss: 0.0232 - acc: 1.0000
677
Epoch 1/2
1/1 [==============================] - 0s 36ms/step - loss: 

Epoch 2/2
1/1 [==============================] - 0s 33ms/step - loss: 1.1666 - acc: 0.6562
315
624
Epoch 1/2
1/1 [==============================] - 0s 33ms/step - loss: 0.4585 - acc: 0.8594
Epoch 2/2
1/1 [==============================] - 0s 34ms/step - loss: 0.4433 - acc: 0.8594
731
Epoch 1/2
1/1 [==============================] - 0s 33ms/step - loss: 0.4093 - acc: 0.8750
Epoch 2/2
1/1 [==============================] - 0s 33ms/step - loss: 0.3989 - acc: 0.8750
425
Epoch 1/2
1/1 [==============================] - 0s 33ms/step - loss: 0.0852 - acc: 1.0000
Epoch 2/2
1/1 [==============================] - 0s 100ms/step - loss: 0.0955 - acc: 1.0000
